In [ ]:
!pip install -q ragas rapidfuzz
!pip install -q sentence_transformers tiktoken lark
!pip install -q langchain langchain-core langchain-google-genai

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)

qwen = ChatOpenAI(
    model="qwen/qwen-2.5-72b-instruct:free",
    temperature=0,
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
    rate_limiter=rate_limiter,
)

/tmp/ipykernel_86754/730842264.py:4: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)
MODEL_NAME = "gemini-2.5-flash-preview-05-20"

gemini_chat = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0, api_key=gemini_api_key, rate_limiter=rate_limiter)
gemini_chat_half = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0.5, api_key=gemini_api_key, rate_limiter=rate_limiter)
gemini_chat_1 = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=1, api_key=gemini_api_key, rate_limiter=rate_limiter)


/tmp/ipykernel_2852/3147785046.py:4: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)


In [3]:
from ragas.llms import LangchainLLMWrapper
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
ragas_llm = LangchainLLMWrapper(qwen)

/tmp/ipykernel_86754/4141154572.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


In [4]:
import pandas as pd

records_df = pd.read_csv('./data/records.csv')

In [5]:
from db.db_manager import FitnessDB

db_manager = FitnessDB(db_path='./fitness_temp1.db', use_dummy_data=False)

SQLite database './fitness_temp1.db' created/verified with updated schema.


In [6]:
from werkzeug.security import generate_password_hash

for i, user in records_df.iterrows():
    db_manager.create_user(
        email = user['email'],
        password_hash = generate_password_hash(user['password']),
        first_name = user['first_name'],
        last_name = user['last_name'],
        date_of_birth = user['date_of_birth'],
        sex =  user['sex'],
        height_cm = user['height_cm'],
        weight_kg = user['weight_kg'],
        activity_level = user['activity_level'],
        fitness_goals = user['fitness_goals'],
    )

In [7]:
from agents.main import MainGraph
from langchain_core.messages import HumanMessage

agent = MainGraph(gemini_chat, db_manager, last_messages=1).compile()

Making the book retriever ready =====
table is already created=====
Book Retriever is ready ====


In [8]:
results = {
    "answers": [],
    "contexts": []
}

for i, user in records_df.iterrows():
    response = agent.invoke({"messages": HumanMessage(content=user['query']), 'user_id':i+1}, config={"configurable": {"thread_id": i+1}})

    results["answers"].append(response["messages"][-1].content)
    results["contexts"].append(response["contexts"])



Function 'node_task_identifier'
Returned: {'task_type': <TaskType.FITNESS: 'fitness'>} 
Function 'node_check_tasks'
Returned: node_plan 
Function 'node_plan'
Function 'node_log_summary'
Function 'node_summarize'
Returned: {'messages': [], 'user_id': 1} 
Returned: {'log_summary': None} 
Function 'node_search_doc'
Returned: {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_books', 'arguments': '{"query": "how often should a beginner lift weights for muscle growth"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b445082a-b216-4488-b27e-e18a29e8a68d-0', tool_calls=[{'name': 'search_books', 'args': {'query': 'beginner weight training program for muscle gain'}, 'id': '42125893-0208-4c79-b052-ed2ca6bd2f56', 'type': 'tool_call'}, {'name': 'search_books', 'args': {'query': 'chest and back exercises for beginners'}, 'id': '6f3d2c8a-0e74-4f76-9b76-e81482e739c1', 'ty

In [9]:
results

{'answers': ["It's fantastic that you're ready to embark on your structured weight training journey, especially with a clear focus on building muscle in your chest and back! This beginner-friendly program is designed to help you establish a strong foundation, learn proper form, and progressively build strength and muscle.\n\nRemember, consistency is key. Aim to perform this workout 2-3 times per week, allowing at least one rest day in between sessions for your muscles to recover and grow. Listen to your body, and don't hesitate to modify exercises as needed.\n\n---\n\n### Beginner Chest & Back Strength Program\n\n**Goal:** Build muscle and increase strength, especially in chest and back.\n**Frequency:** 2-3 times per week (e.g., Monday, Wednesday, Friday)\n**Equipment:** Dumbbells (or bodyweight alternatives), a sturdy bench or floor space. Access to gym machines (like a lat pulldown) is a bonus but not essential for this plan.\n\n---\n\n### Phase 1: Warm-up (5-10 minutes)\n\nThe warm-

In [10]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : records_df['query'].tolist(),
    "answer" : results["answers"],
    "contexts" : results["contexts"],
})


In [12]:
response_dataset.save_to_disk('./data/ragas_dataset')


Saving the dataset (0/1 shards):   0%|          | 0/37 [00:00<?, ? examples/s]

In [4]:
from datasets import load_from_disk
response_dataset = load_from_disk('./data/ragas_dataset')


In [5]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

metrics = [
    faithfulness,
    answer_relevancy,
]

In [ ]:
per_sample = []
for i in range(response_dataset.num_rows):
    result = evaluate(
        dataset=response_dataset.select([i]),
        metrics=metrics,
        llm=ragas_llm,
        embeddings=embeddings,
    )
    per_sample.append(result.scores[0])
    print(f"{i+1}: {result.scores[0]}")


In [ ]:
import pandas as pd

pd.DataFrame(per_sample).to_csv('./data/ragas_eval.csv', index=False)

In [23]:
import pandas as pd

ragas_eval = pd.read_csv("./data/ragas_eval.csv")

In [26]:
ragas_eval.mean()

faithfulness        0.116747
answer_relevancy    0.832424
dtype: float64